This is the tutorial I partially re-implemented from the reference attached below on FHE 
directly using SEAL API with cmake and the computation in Python [1]

In [46]:
import ctypes

In [47]:
seal_lib = ctypes.CDLL("./SEAL/build/lib/libsealc.so.3.7.2")

In [ ]:
seal_lib.EncParams_Create1.argtypes = [ ctypes.c_byte, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.EncParams_SetPlainModulus1.argtypes = [ ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.EncParams_SetPolyModulusDegree.argtypes = [ ctypes.c_void_p, ctypes.c_ulonglong ]
seal_lib.EncParams_SetCoeffModulus.argtypes = [ ctypes.c_void_p, ctypes.c_ulonglong, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.Modulus_Create1.argtypes = [ ctypes.c_ulonglong, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.CoeffModulus_BFVDefault.argtypes = [ ctypes.c_ulonglong, ctypes.c_int, ctypes.POINTER(ctypes.c_ulonglong), ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.SEALContext_Create.argtypes = [ ctypes.c_void_p, ctypes.c_bool, ctypes.c_int, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.KeyGenerator_Create1.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.KeyGenerator_SecretKey.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.KeyGenerator_CreatePublicKey.argtypes = [ ctypes.c_void_p, ctypes.c_bool, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.KeyGenerator_CreateRelinKeys.argtypes = [ ctypes.c_void_p, ctypes.c_bool, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Encryptor_Create.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Encryptor_Encrypt.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Decryptor_Create.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Decryptor_Decrypt.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Plaintext_Create1.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Plaintext_Create4.argtypes = [ ctypes.c_char_p, ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Plaintext_ToString.argtypes = [ ctypes.c_void_p, ctypes.c_char_p, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.Ciphertext_Create1.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Ciphertext_SaveSize.argtypes = [ ctypes.c_void_p, ctypes.c_ubyte, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.Ciphertext_Save.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_ubyte), ctypes.c_ulonglong, ctypes.c_ubyte, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.Ciphertext_Size.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_ulonglong) ]
seal_lib.Serialization_ComprModeDefault.argtypes = [ ctypes.POINTER(ctypes.c_ubyte) ]
seal_lib.Evaluator_Create.argtypes = [ ctypes.c_void_p, ctypes.POINTER(ctypes.c_void_p) ]
seal_lib.Evaluator_Square.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Evaluator_AddPlain.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Evaluator_Multiply.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]
seal_lib.Evaluator_Relinearize.argtypes = [ ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p ]

In [32]:
#
# Create encryption parameter (CKKS)
#

ptr_encparm = ctypes.c_void_p()
seal_lib.EncParams_Create1(
  ctypes.c_byte(0x02), # 0x02 means CKKS
  ctypes.byref(ptr_encparm)
)

#
# Set encryption parameter's attribute
#

# 1. Set PolyModulusDegree
seal_lib.EncParams_SetPolyModulusDegree(
  ptr_encparm,
  ctypes.c_ulonglong(8192)
)
# 2. Create CoeffModulus
# (Here I create 5 40-bits prime numbers.)
coeff_size_arr = (ctypes.c_int * 5)()
coeff_size_arr[0] = 40
coeff_size_arr[1] = 40
coeff_size_arr[2] = 40
coeff_size_arr[3] = 40
coeff_size_arr[4] = 40
coeff_arr = (ctypes.c_ulong * 5)()
seal_lib.CoeffModulus_Create(
  ctypes.c_ulonglong(8192),
  ctypes.c_ulonglong(5),
  ctypes.cast(coeff_size_arr, ctypes.POINTER(ctypes.c_int)),
  ctypes.cast(coeff_arr, ctypes.POINTER(ctypes.c_ulong))
)
# 3. Set CoeffModulus
seal_lib.EncParams_SetCoeffModulus(
  ptr_encparm,
  ctypes.c_ulonglong(5),
  ctypes.cast(coeff_arr, ctypes.POINTER(ctypes.c_ulong))
)

#
# Create SEAL context
# with encryption parameter
#

# Create
ptr_context = ctypes.c_void_p()
seal_lib.SEALContext_Create(
  ptr_encparm,
  ctypes.c_bool(True),
  ctypes.c_int(128),
  ctypes.byref(ptr_context)
)
# Get first parms id
parms_id = ctypes.c_ulonglong()
seal_lib.SEALContext_FirstParmsId(
  ptr_context,
  ctypes.byref(parms_id)
)

0

In [33]:
#
# Create keys for encryption, decryption, and relinearization
#

# Create key generator
ptr_key_generator = ctypes.c_void_p()
seal_lib.KeyGenerator_Create1(
  ptr_context,
  ctypes.byref(ptr_key_generator)
)
# Get secret key used for decryption
# (Use SecretKey_Data() when you show secret key text)
ptr_secret_key = ctypes.c_void_p()
seal_lib.KeyGenerator_SecretKey(
  ptr_key_generator,
  ctypes.byref(ptr_secret_key)
)
# Create public key used for encryption
ptr_public_key = ctypes.c_void_p()
seal_lib.KeyGenerator_CreatePublicKey(
  ptr_key_generator,
  ctypes.c_bool(False),
  ctypes.byref(ptr_public_key)
)
# Create relinearization key used for relinearization
ptr_relin_key = ctypes.c_void_p()
seal_lib.KeyGenerator_CreateRelinKeys(
  ptr_key_generator,
  ctypes.c_bool(False),
  ctypes.byref(ptr_relin_key)
)

0

In [34]:
#
# Create CKKS encoder
#

# Create CKKS encoder
ptr_encoder = ctypes.c_void_p()
seal_lib.CKKSEncoder_Create(
  ptr_context,
  ctypes.byref(ptr_encoder)
)
# In CKKS, the number of slots is PolyModulusDegree / 2 and each slot encodes one number.
# In encryption, the encoder will implicitly pad it with zeros to full size, PolyModulusDegree / 2.
slot_size = ctypes.c_ulonglong()
seal_lib.CKKSEncoder_SlotCount(
  ptr_encoder,
  ctypes.byref(slot_size)
)

#
# Create plaintexts for PI (3.14159265) and 0.5
# with CKKS encoder
# (Encodes these floating-point values to every slot)
#

# Scale S = 2^40
scale = pow(2.0, 40)
# Create plain text for PI
ptr_plain_pi = ctypes.c_void_p()
seal_lib.Plaintext_Create1(
  None,
  ctypes.byref(ptr_plain_pi)
)
# CKKS encode for PI
seal_lib.CKKSEncoder_Encode3(
  ptr_encoder,
  ctypes.c_double(3.14159265),
  ctypes.byref(parms_id),
  ctypes.c_double(scale),
  ptr_plain_pi,
  None
)
# Create plain text for 0.5
ptr_plain_05 = ctypes.c_void_p()
seal_lib.Plaintext_Create1(
  None,
  ctypes.byref(ptr_plain_05)
)
# CKKS encode for 0.5
seal_lib.CKKSEncoder_Encode3(
  ptr_encoder,
  ctypes.c_double(0.5),
  ctypes.byref(parms_id),
  ctypes.c_double(scale),
  ptr_plain_05,
  None
)

0

In [35]:
#
# Create encryption for x (= 0.0, 1.1, 2.2, 3.3)
# with CKKS encoder
# (Other slots will be padded with zeros)
#

# Create plain text for x
ptr_plain_x = ctypes.c_void_p()
seal_lib.Plaintext_Create1(
  None,
  ctypes.byref(ptr_plain_x)
)
# CKKS encode for x
x_arr = (ctypes.c_double * 4)()
x_arr[0] = 0.0
x_arr[1] = 1.1
x_arr[2] = 2.2
x_arr[3] = 3.3
seal_lib.CKKSEncoder_Encode1(
  ptr_encoder,
  ctypes.c_ulonglong(4),
  ctypes.cast(x_arr, ctypes.POINTER(ctypes.c_double)),
  ctypes.byref(parms_id),
  ctypes.c_double(scale),
  ptr_plain_x,
  None
)
# Create cipher text for x
ptr_cipher_x = ctypes.c_void_p()
seal_lib.Ciphertext_Create1(
  None,
  ctypes.byref(ptr_cipher_x)
)
# Create encryptor
ptr_encryptor = ctypes.c_void_p()
seal_lib.Encryptor_Create(
  ptr_context,
  ptr_public_key,
  None,
  ctypes.byref(ptr_encryptor)
)
# Encrypt for x
seal_lib.Encryptor_Encrypt(
  ptr_encryptor,
  ptr_plain_x,
  ptr_cipher_x,
  None
)

0

In [36]:
#
# Compute x^2
#

# Create Evaluator
ptr_evaluator = ctypes.c_void_p()
seal_lib.Evaluator_Create(
  ptr_context,
  ctypes.byref(ptr_evaluator)
)

0

In [37]:
# Create cipher text for result
ptr_cipher_res1 = ctypes.c_void_p()
seal_lib.Ciphertext_Create1(
  None,
  ctypes.byref(ptr_cipher_res1)
)

0

In [38]:
# Square x
seal_lib.Evaluator_Square(
  ptr_evaluator,
  ptr_cipher_x,
  ptr_cipher_res1,
  None
)

0

This is a tutorial re-implementation for CKKS framework [2]

There are some framworks in CKKS that still needs to be researched and cleared such as: 
1. the C[X]/(X^N+1) instead of R=Zq[X]/(X^N+1),, given input vector z = C^(N/2) a polynomial conversion
2. RLWE and LWE for operations: changing from O(n^2) to O(n) complexity
3. ciphertext to cipertext multiplication -> the growth of the computational complexity like the increase in the degree of secret key 
4. relinearization: part of using evaluation key
5. rescaling

with RLWE

addition operation on a single message

secret s, and a public key p, given(b,a)=(−a.s+e,a) = (c0,c1)

For encrypting a message μ we get output c=(μ+b,a)

and to decrypt it with s we evaluate c = c0+c1.s = μ-a.s+e+a.s = μ+e ≈ μ

add: two messages, μ and μ′, encrypt into c=(c0,c1) and c′=(c′0,c′1) respectively 

c_add=c+c′=(c0+c′0,c1+c′1) equivalent to encryption of μ+μ′, i.e. 

implying if we decrypt it using s we get (approximatively) μ+μ′

the decryption c_add,0+cadd,1.s=c0+c′0+(c1+c′1).s=(c0+c1.s)+(c′0+c′1.s)= μ-a.s+e+a.s + μ'-a.s+e+a.s = μ+μ′+2e≈μ+μ′

multiplication of ciphertext with plaintext

plaintext μ, encrypted into the ciphertext c=(c0,c1) and a plaintext μ′

ciphertext of the multiplication, need output c_mult=(μ′.c0,μ′.c1)

decrypting c_mult we get μ′.c0+μ′.c1.s=μ′.(c0+c1.s)= μ′.(μ-a.s+e+a.s) = μ′.(μ+e)=μ′.μ+μ′.e≈μ′.μ

In [2]:
import numpy as np

# First we set the parameters
M = 8
N = M //2

# We set xi, which will be used in our computations
xi = np.exp(2 * np.pi * 1j / M)
xi

(0.7071067811865476+0.7071067811865475j)

In [4]:
from numpy.polynomial import Polynomial

class CKKSEncoder:
    """Basic CKKS encoder to encode complex vectors into polynomials."""
    
    def __init__(self, M: int):
        """Initialization of the encoder for M a power of 2. 
        
        xi, which is an M-th root of unity will, be used as a basis for our computations.
        """
        self.xi = np.exp(2 * np.pi * 1j / M)
        self.M = M
        
    @staticmethod
    def vandermonde(xi: np.complex128, M: int) -> np.array:
        """Computes the Vandermonde matrix from a m-th root of unity."""
        
        N = M //2
        matrix = []
        # We will generate each row of the matrix
        for i in range(N):
            # For each row we select a different root
            root = xi ** (2 * i + 1)
            row = []

            # Then we store its powers
            for j in range(N):
                row.append(root ** j)
            matrix.append(row)
        return matrix
    
    def sigma_inverse(self, b: np.array) -> Polynomial:
        """Encodes the vector b in a polynomial using an M-th root of unity."""

        # First we create the Vandermonde matrix
        A = CKKSEncoder.vandermonde(self.xi, M)

        # Then we solve the system
        coeffs = np.linalg.solve(A, b)

        # Finally we output the polynomial
        p = Polynomial(coeffs)
        return p

    def sigma(self, p: Polynomial) -> np.array:
        """Decodes a polynomial by applying it to the M-th roots of unity."""

        outputs = []
        N = self.M //2

        # We simply apply the polynomial on the roots
        for i in range(N):
            root = self.xi ** (2 * i + 1)
            output = p(root)
            outputs.append(output)
        return np.array(outputs)

In [5]:
# First we initialize our encoder
encoder = CKKSEncoder(M)

b = np.array([1, 2, 3, 4])
b

array([1, 2, 3, 4])

In [6]:
p = encoder.sigma_inverse(b)
p

Polynomial([ 2.50000000e+00+4.44089210e-16j, -4.99600361e-16+7.07106781e-01j,
       -3.46944695e-16+5.00000000e-01j, -8.32667268e-16+7.07106781e-01j], domain=[-1,  1], window=[-1,  1], symbol='x')

In [7]:
b_reconstructed = encoder.sigma(p)
b_reconstructed

array([1.-1.11022302e-16j, 2.-4.71844785e-16j, 3.+2.77555756e-17j,
       4.+2.22044605e-16j])

In [8]:
np.linalg.norm(b_reconstructed - b)

6.944442800358888e-16

In [9]:
m1 = np.array([1, 2, 3, 4])
m2 = np.array([1, -2, 3, -4])

p1 = encoder.sigma_inverse(m1)
p2 = encoder.sigma_inverse(m2)

In [10]:
p_add = p1 + p2
p_add

Polynomial([ 2.00000000e+00+1.11022302e-16j, -7.07106781e-01+7.07106781e-01j,
        2.10942375e-15-2.00000000e+00j,  7.07106781e-01+7.07106781e-01j], domain=[-1.,  1.], window=[-1.,  1.], symbol='x')

In [11]:
encoder.sigma(p_add)

array([2.0000000e+00+3.25176795e-17j, 4.4408921e-16-4.44089210e-16j,
       6.0000000e+00+1.11022302e-16j, 4.4408921e-16+3.33066907e-16j])

In [12]:
poly_modulo = Polynomial([1,0,0,0,1])
poly_modulo

Polynomial([1., 0., 0., 0., 1.], domain=[-1,  1], window=[-1,  1], symbol='x')

In [13]:
p_mult = p1 * p2 % poly_modulo

In [14]:
encoder.sigma(p_mult)

array([  1.-8.67361738e-16j,  -4.+6.86950496e-16j,   9.+6.86950496e-16j,
       -16.-9.08301212e-15j])

In [15]:
from fastcore.foundation import patch_to

In [16]:
@patch_to(CKKSEncoder)
def pi(self, z: np.array) -> np.array:
    """Projects a vector of H into C^{N/2}."""
    
    N = self.M // 4
    return z[:N]

@patch_to(CKKSEncoder)
def pi_inverse(self, z: np.array) -> np.array:
    """Expands a vector of C^{N/2} by expanding it with its
    complex conjugate."""
    
    z_conjugate = z[::-1]
    z_conjugate = [np.conjugate(x) for x in z_conjugate]
    return np.concatenate([z, z_conjugate])

# We can now initialize our encoder with the added methods
encoder = CKKSEncoder(M)

In [17]:
z = np.array([0,1])
encoder.pi_inverse(z)

array([0, 1, 1, 0])

In [19]:
@patch_to(CKKSEncoder)
def create_sigma_R_basis(self):
    """Creates the basis (sigma(1), sigma(X), ..., sigma(X** N-1))."""

    self.sigma_R_basis = np.array(self.vandermonde(self.xi, self.M)).T
    
@patch_to(CKKSEncoder)
def __init__(self, M):
    """Initialize with the basis"""
    self.xi = np.exp(2 * np.pi * 1j / M)
    self.M = M
    self.create_sigma_R_basis()
    
encoder = CKKSEncoder(M)

In [20]:
encoder.sigma_R_basis

array([[ 1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ,
         1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ],
       [ 7.07106781e-01+0.70710678j, -7.07106781e-01+0.70710678j,
        -7.07106781e-01-0.70710678j,  7.07106781e-01-0.70710678j],
       [ 2.22044605e-16+1.j        , -4.44089210e-16-1.j        ,
         1.11022302e-15+1.j        , -1.38777878e-15-1.j        ],
       [-7.07106781e-01+0.70710678j,  7.07106781e-01+0.70710678j,
         7.07106781e-01-0.70710678j, -7.07106781e-01-0.70710678j]])

In [21]:
# Here we simply take a vector whose coordinates are (1,1,1,1) in the lattice basis
coordinates = [1,1,1,1]

b = np.matmul(encoder.sigma_R_basis.T, coordinates)
b

array([1.+2.41421356j, 1.+0.41421356j, 1.-0.41421356j, 1.-2.41421356j])

In [22]:
@patch_to(CKKSEncoder)
def compute_basis_coordinates(self, z):
    """Computes the coordinates of a vector with respect to the orthogonal lattice basis."""
    output = np.array([np.real(np.vdot(z, b) / np.vdot(b,b)) for b in self.sigma_R_basis])
    return output

def round_coordinates(coordinates):
    """Gives the integral rest."""
    coordinates = coordinates - np.floor(coordinates)
    return coordinates

def coordinate_wise_random_rounding(coordinates):
    """Rounds coordinates randonmly."""
    r = round_coordinates(coordinates)
    f = np.array([np.random.choice([c, c-1], 1, p=[1-c, c]) for c in r]).reshape(-1)
    
    rounded_coordinates = coordinates - f
    rounded_coordinates = [int(coeff) for coeff in rounded_coordinates]
    return rounded_coordinates

@patch_to(CKKSEncoder)
def sigma_R_discretization(self, z):
    """Projects a vector on the lattice using coordinate wise random rounding."""
    coordinates = self.compute_basis_coordinates(z)
    
    rounded_coordinates = coordinate_wise_random_rounding(coordinates)
    y = np.matmul(self.sigma_R_basis.T, rounded_coordinates)
    return y

encoder = CKKSEncoder(M)

In [23]:
@patch_to(CKKSEncoder)
def __init__(self, M:int, scale:float):
    """Initializes with scale."""
    self.xi = np.exp(2 * np.pi * 1j / M)
    self.M = M
    self.create_sigma_R_basis()
    self.scale = scale
    
@patch_to(CKKSEncoder)
def encode(self, z: np.array) -> Polynomial:
    """Encodes a vector by expanding it first to H,
    scale it, project it on the lattice of sigma(R), and performs
    sigma inverse.
    """
    pi_z = self.pi_inverse(z)
    scaled_pi_z = self.scale * pi_z
    rounded_scale_pi_zi = self.sigma_R_discretization(scaled_pi_z)
    p = self.sigma_inverse(rounded_scale_pi_zi)
    
    # We round it afterwards due to numerical imprecision
    coef = np.round(np.real(p.coef)).astype(int)
    p = Polynomial(coef)
    return p

@patch_to(CKKSEncoder)
def decode(self, p: Polynomial) -> np.array:
    """Decodes a polynomial by removing the scale, 
    evaluating on the roots, and project it on C^(N/2)"""
    rescaled_p = p / self.scale
    z = self.sigma(rescaled_p)
    pi_z = self.pi(z)
    return pi_z

scale = 64

encoder = CKKSEncoder(M, scale)

In [24]:
z = np.array([3 +4j, 2 - 1j])
z

array([3.+4.j, 2.-1.j])

In [25]:
p = encoder.encode(z)
p

Polynomial([160.,  91., 160.,  45.], domain=[-1,  1], window=[-1,  1], symbol='x')

In [26]:
encoder.decode(p)

array([3.008233+4.00260191j, 1.991767-0.99739809j])

ML: 
This is the tutorial re-implementation of FHE for tensors [3]

In [1]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
import tenseal as ts

In [2]:
torch.manual_seed(73)

train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST('data', train=False, download=True, transform=transforms.ToTensor())

batch_size = 64

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [3]:
class ConvNet(torch.nn.Module):
    def __init__(self, hidden=64, output=10):
        super(ConvNet, self).__init__()        
        self.conv1 = torch.nn.Conv2d(1, 4, kernel_size=7, padding=0, stride=3)
        self.fc1 = torch.nn.Linear(256, hidden)
        self.fc2 = torch.nn.Linear(hidden, output)

    def forward(self, x):
        x = self.conv1(x)
        # the model uses the square activation function
        x = x * x
        # flattening while keeping the batch axis
        x = x.view(-1, 256)
        x = self.fc1(x)
        x = x * x
        x = self.fc2(x)
        return x


def train(model, train_loader, criterion, optimizer, n_epochs=10):
    # model in training mode
    model.train()
    for epoch in range(1, n_epochs+1):

        train_loss = 0.0
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # calculate average losses
        train_loss = train_loss / len(train_loader)

        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))
    
    # model in evaluation mode
    model.eval()
    return model

In [4]:
model = ConvNet()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = train(model, train_loader, criterion, optimizer, 10)

Epoch: 1 	Training Loss: 0.397561
Epoch: 2 	Training Loss: 0.130699
Epoch: 3 	Training Loss: 0.088399
Epoch: 4 	Training Loss: 0.071318
Epoch: 5 	Training Loss: 0.058989
Epoch: 6 	Training Loss: 0.050542
Epoch: 7 	Training Loss: 0.044438
Epoch: 8 	Training Loss: 0.038264
Epoch: 9 	Training Loss: 0.034633
Epoch: 10 	Training Loss: 0.031598


In [5]:
def test(model, test_loader, criterion):
    # initialize lists to monitor test loss and accuracy
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    # model in evaluation mode
    model.eval()

    for data, target in test_loader:
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item()
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        # calculate test accuracy for each object class
        for i in range(len(target)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # calculate and print avg test loss
    test_loss = test_loss/len(test_loader)
    print(f'Test Loss: {test_loss:.6f}\n')

    for label in range(10):
        print(
            f'Test Accuracy of {label}: {int(100 * class_correct[label] / class_total[label])}% '
            f'({int(np.sum(class_correct[label]))}/{int(np.sum(class_total[label]))})'
        )

    print(
        f'\nTest Accuracy (Overall): {int(100 * np.sum(class_correct) / np.sum(class_total))}% ' 
        f'({int(np.sum(class_correct))}/{int(np.sum(class_total))})'
    )
    
test(model, test_loader, criterion)

Test Loss: 0.077313

Test Accuracy of 0: 98% (968/980)
Test Accuracy of 1: 99% (1130/1135)
Test Accuracy of 2: 98% (1015/1032)
Test Accuracy of 3: 99% (1002/1010)
Test Accuracy of 4: 98% (969/982)
Test Accuracy of 5: 98% (882/892)
Test Accuracy of 6: 98% (946/958)
Test Accuracy of 7: 98% (1008/1028)
Test Accuracy of 8: 97% (945/974)
Test Accuracy of 9: 96% (975/1009)

Test Accuracy (Overall): 98% (9840/10000)


In [6]:
class EncConvNet:
    def __init__(self, torch_nn):
        self.conv1_weight = torch_nn.conv1.weight.data.view(
            torch_nn.conv1.out_channels, torch_nn.conv1.kernel_size[0],
            torch_nn.conv1.kernel_size[1]
        ).tolist()
        self.conv1_bias = torch_nn.conv1.bias.data.tolist()
        
        self.fc1_weight = torch_nn.fc1.weight.T.data.tolist()
        self.fc1_bias = torch_nn.fc1.bias.data.tolist()
        
        self.fc2_weight = torch_nn.fc2.weight.T.data.tolist()
        self.fc2_bias = torch_nn.fc2.bias.data.tolist()
        
        
    def forward(self, enc_x, windows_nb):
        # conv layer
        enc_channels = []
        for kernel, bias in zip(self.conv1_weight, self.conv1_bias):
            y = enc_x.conv2d_im2col(kernel, windows_nb) + bias
            enc_channels.append(y)
        # pack all channels into a single flattened vector
        enc_x = ts.CKKSVector.pack_vectors(enc_channels)
        # square activation
        enc_x.square_()
        # fc1 layer
        enc_x = enc_x.mm(self.fc1_weight) + self.fc1_bias
        # square activation
        enc_x.square_()
        # fc2 layer
        enc_x = enc_x.mm(self.fc2_weight) + self.fc2_bias
        return enc_x
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    
def enc_test(context, model, test_loader, criterion, kernel_shape, stride):
    # initialize lists to monitor test loss and accuracy
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    for data, target in test_loader:
        # Encoding and encryption
        x_enc, windows_nb = ts.im2col_encoding(
            context, data.view(28, 28).tolist(), kernel_shape[0],
            kernel_shape[1], stride)
        # Encrypted evaluation
        enc_output = enc_model(x_enc, windows_nb)
        # Decryption of result
        output = enc_output.decrypt()
        output = torch.tensor(output).view(1, -1)

        # compute loss
        loss = criterion(output, target)
        test_loss += loss.item()
        
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        # calculate test accuracy for each object class
        label = target.data[0]
        class_correct[label] += correct.item()
        class_total[label] += 1


    # calculate and print avg test loss
    test_loss = test_loss / sum(class_total)
    print(f'Test Loss: {test_loss:.6f}\n')

    for label in range(10):
        print(
            f'Test Accuracy of {label}: {int(100 * class_correct[label] / class_total[label])}% '
            f'({int(np.sum(class_correct[label]))}/{int(np.sum(class_total[label]))})'
        )

    print(
        f'\nTest Accuracy (Overall): {int(100 * np.sum(class_correct) / np.sum(class_total))}% ' 
        f'({int(np.sum(class_correct))}/{int(np.sum(class_total))})'
    )
    
# Load one element at a time
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)
# required for encoding
kernel_shape = model.conv1.kernel_size
stride = model.conv1.stride[0]

In [51]:
## Encryption Parameters

# controls precision of the fractional part
#bits_scale = 21

# Create TenSEAL context
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,#must be a power of 2 (e.g. 1024,2048,4096,8192, 16384..)
    coeff_mod_bit_sizes=[25, 21, 21, 21, 21, 21, 21, 25]
)
# I am still trying to understand the dependency of modulus degree and mod bit sizes with bits scale on the computational
#speed and precision for 128-bit security
# set the scale
context.global_scale = pow(2, 21)
#rescaling: suppose we must do L multiplications, with a scale Δ, then we will define q as:q=Δ^L.q0
#if we suppose we want 11 bits of precision for the decimal part, and 10 bits of precision for the integer part, 
#we will set:Δ=2^11,q0=2^(# bits integer).2^(# bits decimal)=2^(10+11)=2^21, q -> modulo of the coefficients of polynomial
#Rq=Zq[X]/(X^N+1)

# galois keys are required to do ciphertext rotations
context.generate_galois_keys()

In [52]:
#took around 0.75 to 1 hour and changes depend on context parameters.
enc_model = EncConvNet(model)
enc_test(context, enc_model, test_loader, criterion, kernel_shape, stride)

Test Loss: 116.512219

Test Accuracy of 0: 61% (606/980)
Test Accuracy of 1: 93% (1057/1135)
Test Accuracy of 2: 50% (516/1032)
Test Accuracy of 3: 73% (742/1010)
Test Accuracy of 4: 63% (621/982)
Test Accuracy of 5: 55% (495/892)
Test Accuracy of 6: 69% (662/958)
Test Accuracy of 7: 74% (765/1028)
Test Accuracy of 8: 74% (727/974)
Test Accuracy of 9: 66% (671/1009)

Test Accuracy (Overall): 68% (6862/10000)


Reference:

[1] https://tsmatz.wordpress.com/2022/01/26/microsoft-seal-homomorphic-encryption-ml/ 

[2] https://blog.openmined.org/

[3] https://github.com/OpenMined/TenSEAL